In [ ]:
import scanpy as sc
import scTenifoldXct as st
from scTenifoldXct.cko import get_cko_data


import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [ ]:
# get normal sample
adata = sc.read_h5ad('../data/adata_merge_example.h5ad')
ada_wt = adata[adata.obs['NormalvsTumor'] == 'N', :].copy()
print(ada_wt.shape)

(101, 2608)


In [ ]:
ada_ko = get_cko_data(ada_wt, ['HES4', 'CCNL2'])
print(ada_ko.shape)   

CKO genes ['hes4', 'ccnl2'] are set
(101, 2608)


In [ ]:
xct_wt = st.scTenifoldXct(data = ada_wt, 
                          source_celltype = 'B cells', 
                          target_celltype = 'Fibroblasts',
                          obs_label = "ident",
                          rebuild_GRN = True,
                          GRN_file_dir = 'Net_B2Fib_N/',  
                          verbose = True,
                          n_cpus = 4,
                          )

(cell, feature): (65, 2608)
(cell, feature): (36, 2608)
selected 64 LR pairs
load GRN B cells
load GRN Fibroblasts
build correspondence and initiate a trainer
scTenifoldXct init completed



In [ ]:
xct_ko = st.scTenifoldXct(data = ada_ko, 
                          source_celltype = 'B cells', 
                          target_celltype = 'Fibroblasts',
                          obs_label = "ident",
                          rebuild_GRN = True, 
                          GRN_file_dir = 'Net_B2Fib_CKO/',  
                          verbose = True,
                          n_cpus = 4,
                          )

(cell, feature): (65, 2608)
(cell, feature): (36, 2608)
selected 64 LR pairs
building GRN of B cells...


2025-03-22 17:06:43,081	INFO worker.py:1518 -- Started a local Ray instance.


ray init, using 4 CPUs
execution time of making pcNet: 12.00 s
building GRN of Fibroblasts...


2025-03-22 17:06:54,995	INFO worker.py:1518 -- Started a local Ray instance.


ray init, using 4 CPUs
execution time of making pcNet: 10.78 s
build correspondence and initiate a trainer
scTenifoldXct init completed



In [20]:
xcts = st.merge_scTenifoldXct(xct_ko, xct_wt)

merge samples and build correspondence
merge_scTenifoldXct init completed



In [21]:
emb = xcts.get_embeds(train = True)
emb.shape

training...: 100%|██████████| 3000/3000 [21:47<00:00,  2.29it/s]


(10432, 3)

In [22]:
xcts.nn_aligned_diff(emb) 
xcts_pairs_diff = xcts.chi2_diff_test()
xcts_pairs_diff

computing pair-wise euclidean distances...
computing pair-wise euclidean distances...
merged pair-wise distances

Total enriched: 2 / 6801664


,ligand,receptor,dist,correspondence,dist,correspondence,diff2,FC,p_val,q_val,enriched_rank
ORM1_F3,ORM1,F3,0.014251,0.148032,0.014717,0.14757,2.173538e-07,40.714890,1.761399e-10,5.765372e-07,1
IFNG_CXCL9,IFNG,CXCL9,0.014291,0.148032,0.014580,0.14757,8.357626e-08,15.655571,7.598841e-05,1.817578e-02,2
